%matplotlib inline

import matplotlib
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob

from geopy.distance import vincenty


import seaborn as sns
sns.set()

## Load Trip Data

In [ ]:
print('Loading Trip Data...')

try:
    file_path_slug = '../../datasets/bayareabikeshare/*_trip_data.csv'

    # glob all files
    file_list = glob(file_path_slug)

    trip_import = pd.DataFrame()

    counter = 1
    chunks = []

    # load data from each file
    for file in file_list:

        # import file in chunks
        for chunk in pd.read_csv(file, chunksize=10000, iterator=True):

            # set chunk index column to 'Trip ID'
            chunk = chunk.set_index('Trip ID')

            # define Columns
            chunk.columns = ['Duration', 'Start Date', 'Start Station', 'Start Terminal', 'End Date', 
                             'End Station', 'End Terminal', 'Bike #', 'Subscriber Type', 'Zip Code']

            # append chunk to chunks list
            chunks.append(chunk)

        print('\tFinished file! (%d of %d)' % (counter, len(file_list)))
        counter += 1

    # concat chunks
    trip_import = pd.concat(chunks)

    print('Data Loaded Successfully!')

except:
    print('oops... something went wrong importing the data :(')

### Clean Trip Data

In [ ]:
trip_data = trip_import.copy()
trip_data.info()

In [ ]:
# our data set show duration in seconds, here are some handy conversions
second = 1
minute = second * 60
hour = minute * 60

# station ID numbers that are in San Francisco
sf_stations = [ 39,41,42,45,46,47,48,49,50,51,54,55,56,57,58,59,60,61,62,63,
                64,65,66,67,68,69,70,71,72,73,74,75,76,77,82,90,91]

In [ ]:
# zipcodes are all over the place, only keep corrected 5 digit zipcodes, and replace all others with NaNs
def clean_zipcode(item):
    if len(item) != 5:

        # split on '-'
        try:
            result = item.split('-')[0]
        except:
            result = item

        # split on '.'
        try:
            result = item.split('.')[0]
        except:
            result = item
        
        # if len of item is less than 5, return 'NaN'
        if len(result) < 5:
            result = 'NaN'
        else:
            # if len result is greater than 5, take at most, first 5 digits
            result = result[:5]
    else:
        result = item
    
    # make sure result is all digits
    if result.isdigit():
        return result
    else:
        return 'NaN'

In [ ]:
print('Trip Data Cleanup Started...')

# cleanup column names
print('\tcleaning column names')
new_cols = []
for col in trip_data.columns:
    new_cols.append(col.replace(' ', '_').lower())
trip_data.columns = new_cols

# extract columns we want to keep
print('\tsubsetting to useful columns')
important_cols = ['duration', 'start_date', 'start_terminal', 'end_date', 'end_terminal', 'bike_#', 'subscriber_type', 'zip_code']
trip_data = trip_data[important_cols]

# we are only looking at stations in San Francisco
sf_trips_data = trip_data[trip_data['start_terminal'].isin(sf_stations)]
sf_trips_data = sf_trips_data[sf_trips_data['end_terminal'].isin(sf_stations)]

trip_data = sf_trips_data.copy()

# create duration minutes column
print('\tcreating a duration_minutes column')
trip_data['duration_minutes'] = trip_data['duration'] / 60.0

# convert end and start dates to datetime objects
print('\tconverting end and start dates to datetime objects')
trip_data['start_date'] = pd.to_datetime(trip_data['start_date'], format="%m/%d/%Y %H:%M")
trip_data['end_date']   = pd.to_datetime(trip_data['end_date'],   format="%m/%d/%Y %H:%M")

# create a start and end hour trip column
print('\tcreating trip_date and trip_dow columns')
trip_data['trip_date']  = trip_data['start_date'].dt.date
trip_data['trip_dow']  = trip_data['start_date'].dt.weekday
trip_data['trip_day']  = trip_data['start_date'].dt.weekday_name

print('\tcreating start_hour and end_hour columns')
trip_data['start_hour'] = trip_data['start_date'].dt.hour
trip_data['end_hour']   = trip_data['end_date'].dt.hour

# convert and clean zipcodes
print('\tcleaning zipcodes')
trip_data['zip_code'] = trip_data['zip_code'].astype(str)
trip_data.zip_code = trip_data.zip_code.apply(clean_zipcode)
trip_data['zip_code'] = pd.to_numeric(trip_data['zip_code'], errors='coerce')

# clean up data types
print('cleaning up data types')

trip_data['duration']         = trip_data['duration'].astype('float')
# trip_data['start_date']       = trip_data['start_date'].astype('')
trip_data['start_terminal']   = trip_data['start_terminal'].astype('category')
# trip_data['end_date']         = trip_data['end_date'].astype('')
trip_data['end_terminal']     = trip_data['end_terminal'].astype('category')
trip_data['bike_#']           = trip_data['bike_#'].astype('int')
trip_data['subscriber_type']  = trip_data['subscriber_type'].astype('category')
trip_data['zip_code']         = trip_data['zip_code'].astype('str')
trip_data['duration_minutes'] = trip_data['duration_minutes'].astype('float')
# trip_data['trip_date']        = trip_data['trip_date'].astype('')
trip_data['trip_dow']         = trip_data['trip_dow'].astype('category')
trip_data['trip_day']         = trip_data['trip_day'].astype('category')
# trip_data['start_hour']       = trip_data['start_hour'].astype('')
# trip_data['end_hour']         = trip_data['end_hour'].astype('')


print('Trip Data Cleanup complete')
trip_clean = trip_data.copy()

In [ ]:
trip_clean.info()

In [ ]:
# prune data to exclude trips longer than 60 minutes
print('pruning data to trips no more than 60 minutes long...')
trips = trip_clean[trip_clean['duration_minutes'] <= 60].copy()

# Cleanup
trips.sort_index(inplace=True)
print('\tpruned data set \'trips\' consists of %i entries' % len(trips.index))

plt.subplots(figsize=(12,6))
ax = sns.distplot(trips[trips.subscriber_type == 'Subscriber'].start_date.dt.hour, color='b', label='Subscribers')
sns.distplot(trips[trips.subscriber_type == 'Customer'].start_date.dt.hour, color='r', label='Customers', ax=ax)
ax.set(xlabel='start hour')
plt.legend()
plt.show()

## Load Weather Data

In [ ]:
print('Loading Weather Data...')

try:
    file_path_slug = '../../datasets/bayareabikeshare/*_weather_data.csv'

    # glob all files
    file_list = glob(file_path_slug)

    weather_import = pd.DataFrame()

    counter = 1
    chunks = []

    # load data from each file
    for file in file_list:

        # import file in chunks
        for chunk in pd.read_csv(file, chunksize=10000, iterator=True):

            chunk.columns = ['Date', 'Max_Temperature_F', 'Mean_Temperature_F', 'Min_TemperatureF', 'Max_Dew_Point_F', 
                             'MeanDew_Point_F', 'Min_Dewpoint_F', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
                             'Max_Sea_Level_Pressure_In', 'Mean_Sea_Level_Pressure_In', 'Min_Sea_Level_Pressure_In', 
                             'Max_Visibility_Miles', 'Mean_Visibility_Miles', 'Min_Visibility_Miles', 
                             'Max_Wind_Speed_MPH', 'Mean_Wind_Speed_MPH', 'Max_Gust_Speed_MPH', 'Precipitation_In', 
                             'Cloud_Cover', 'Events', 'Wind_Dir_Degrees', 'zip']

            chunks.append(chunk)

        print('\tfinished file! (%d of %d)'% (counter, len(file_list)))
        counter += 1

    # concat chunks
    weather_import = pd.concat(chunks)

    print('Data Loaded Successfully!')
except:
    print('oops... something went wrong loading the data :()')

In [ ]:
weather_data = weather_import.copy()
weather_data.info()

### Clean Weather Data

In [ ]:
print('Weather Data Cleanup Started...')

# cleanup column names
print('\tcleaning column names')
new_cols = []
for col in weather_data.columns:
    new_cols.append(col.replace(' ', '_').lower())
weather_data.columns = new_cols

# convert end and start dates to datetime objects
print('\tconverting dates to datetime objects')
weather_data['date'] = pd.to_datetime(weather_data['date'], format="%m/%d/%Y")

# extract columns we want to keep
print('\tsubsetting to useful columns')
important_cols = ['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperaturef',
                  'max_dew_point_f', 'meandew_point_f', 'min_dewpoint_f',
                  'max_wind_speed_mph', 'mean_wind_speed_mph', 'max_gust_speed_mph',
                  'precipitation_in', 'cloud_cover', 'events', 'zip']
weather_data = weather_data[important_cols]

# correct min_temperaturef column name to min_temperature_f
# weather['min_temperaturef'].rename('min_temperature_f')
weather_data.rename(columns={'min_temperaturef': 'min_temperature_f'}, inplace=True)

# cleanup and set date as index
weather_data.set_index('date', inplace=True)
weather_data.sort_index(inplace=True)

# cleanup precipitation data to be all float values
weather_data['precipitation_in'] = pd.to_numeric(weather_data['precipitation_in'], errors='coerce')

# we only want San Francisco Weather information, zipcode 94107
weather_data = weather_data[weather_data.zip == 94107]

print('Weather Data Cleanup complete')
weather_clean = weather_data.copy()

## Load Station Data

In [ ]:
print('Loading Station Data...')

try:
    file_path_slug = '../../datasets/bayareabikeshare/*_station_data.csv'

    # glob all files
    file_list = glob(file_path_slug)

    station_import = pd.DataFrame()

    counter = 1
    chunks = []

    # load data from each file
    for file in file_list:

        # import file in chunks
        for chunk in pd.read_csv(file, chunksize=10000, iterator=True):

            # set chunk index column to 'Trip ID'
#             chunk = chunk.set_index('Trip ID')

            # define Columns
            chunk.columns = ['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'installation']            
            
            # append chunk to chunks list
            chunks.append(chunk)

        print('\tFinished file! (%d of %d)' % (counter, len(file_list)))
        counter += 1

    # concat chunks
    station_import = pd.concat(chunks)

    print('Data Loaded Successfully!')

except:
    print('oops... something went wrong importing the data :(')

### Clean Station Data

In [ ]:
def label_zip(row):
    if row['landmark'] == 'San Francisco':
       return '94107'
    if row['landmark'] == 'Redwood City':
        return '94063'
    if row['landmark'] == 'Palo Alto':
        return '94301'
    if row['landmark'] == 'Mountain View':
        return '94041'
    if row['landmark'] == 'San Jose':
        return '95113'
    return '99999'

def make_lat_long(row):
    lat = row['lat']
    long = row['long']
    return (lat, long)

In [ ]:
station_data = station_import.copy()

# remove dulplicates
print('remove dulplicates')
station_data.drop_duplicates(keep='first', inplace=True)
station_data.dropna(how='all', inplace=True)

# set datatype for each column
print('set datatype for each column')
station_data['station_id']   = station_data['station_id'].astype('int')
station_data['name']         = station_data['name'].astype('str')
station_data['lat']          = station_data['lat'].astype('float')
station_data['long']         = station_data['long'].astype('float')
station_data['landmark']     = station_data['landmark'].astype('category')

# add a zipcode column for later comparison with weather data
station_data['zip_code'] = station_data.apply(lambda row: label_zip (row),axis=1)
# station_data['zip_code'] = station_data['landmark'].astype('str')

# create lat,lon tuple column
station_data['lat_long'] = station_data.apply(lambda row: make_lat_long (row),axis=1)

# reindex to remove some extra duplicate
print('correcting index')
station_data.reset_index(inplace=True)
station_data.drop_duplicates(['station_id', 'installation'], keep='first', inplace=True)
station_data.set_index('station_id', inplace=True)
station_data.sort_index(inplace=True)
del station_data['index']

station_clean = station_data.copy()
print('Cleaning complete!')
station_clean.info()

## Append Route Distance column to Trips Data

In [ ]:
def route_distance(row):
    
    # round trips are defaulting to zero km
    if row['start_terminal'] == row['end_terminal']:
        dist = 0.0
        return dist
    
#     print(row['start_terminal'], ' ', row['end_terminal'], ' ', row['trip_date'])
    # lookup start_station id coords
    start_gps = station_clean.loc[row['start_terminal']]['lat_long']

    # lookup end_station id coords
    end_gps = station_clean.loc[row['end_terminal']]['lat_long']

    
    '''
        this is sloppy, but some stations moved over time, we are just going to take the most recent 
        location coordinates, this is not an exact measurment by roadways, and we dont know the routes 
        that riders took anyway...
    '''
    

    
    if isinstance(start_gps, pd.core.series.Series):
        start_gps = start_gps.iloc[-1]
    if isinstance(end_gps, pd.core.series.Series):
        end_gps = end_gps.iloc[-1]

    try:
        dist = str(vincenty(start_gps, end_gps))
        dist = float(dist.split(' ')[0])
    except:
        dist = 'NaN'
            
    return dist
    

In [ ]:
trips['distance_km'] = trips.apply(lambda row: route_distance (row),axis=1)
trips.head(3)

In [ ]:
# histogram of ride distances

# plt.subplots(figsize=(12,6))
print(len(trips))

one_way_trips = trips[trips['distance_km'] > 0.0]
one_way_trips = one_way_trips[one_way_trips['duration_minutes'] <= 60.0]


print(len(one_way_trips))

In [ ]:
print('one_way_trips[\'distance_km\'].mean()\t', one_way_trips['distance_km'].mean())
print('one_way_trips[\'distance_km\'].median()\t', one_way_trips['distance_km'].median())
print('one_way_trips[\'distance_km\'].min()\t', one_way_trips['distance_km'].min())
print('one_way_trips[\'distance_km\'].max()\t', one_way_trips['distance_km'].max())

In [ ]:
plt.subplots(figsize=(12,6))
one_way_trips['distance_km'].plot(kind='hist', bins=60)
plt.show()

In [ ]:
# trip_copy_select = trip_copy.head(100000)
plt.subplots(figsize=(15,10))
sns.boxplot(x='start_hour', y='distance_km', data=trips);
plt.show()

In [ ]:
# trips_select_c = trips[trips['subscriber_type'] == 'Customer'].head(20000)
# sns.jointplot('start_terminal', 'distance_km', data=trips_select_c, kind='reg')
# plt.show()

# trips_select_s = trips[trips['subscriber_type'] == 'Subscriber'].head(20000)
# sns.jointplot('start_terminal', 'distance_km', data=trips_select_s, kind='reg')
# plt.show()

In [ ]:
print('Customers:\t ', len(one_way_trips[one_way_trips['subscriber_type'] == 'Customer']))
print('Subscribers:\t', len(one_way_trips[one_way_trips['subscriber_type'] != 'Customer']))

In [ ]:
# sns.jointplot('duration_minutes', 'distance_km', data=one_way_trips[one_way_trips['subscriber_type'] == 'Customer'], kind='reg')
# plt.legend()
# plt.show()

In [ ]:
# sns.jointplot('duration_minutes', 'distance_km', data=one_way_trips[one_way_trips['subscriber_type'] == 'Subscriber'], kind='reg')
# plt.legend()
# plt.show()

In [ ]:
with sns.axes_style("white"):
    sns.jointplot(x=one_way_trips['duration_minutes'], y=one_way_trips['distance_km'], kind="hex", color="k");

# Analysis

## 1. Does Rain have an Impact on trip duration for Customers, or Subscribers?

In [ ]:
# trips.info()

In [ ]:
# split up rainy days and dry days
rainy_days = weather_clean[weather_clean['precipitation_in'] > 0.0].reset_index()
dry_days = weather_clean[weather_clean['precipitation_in'] == 0.0].reset_index()

# All trips
rainy_trips = trips[trips['start_date'].dt.date.isin(rainy_days['date'].dt.date)]
dry_trips = trips[trips['start_date'].dt.date.isin(dry_days['date'].dt.date)]

# Customer Trips
customer_rainy_trips = rainy_trips[rainy_trips.subscriber_type == 'Customer']
customer_dry_trips = dry_trips[dry_trips.subscriber_type == 'Customer']

# Subscriber Trips
subscriber_rainy_trips = rainy_trips[rainy_trips.subscriber_type == 'Subscriber']
subscriber_dry_trips = dry_trips[dry_trips.subscriber_type == 'Subscriber']

> A <b>Two Sample T Test</b> is appropriate for this problem as we are trying to see a difference between two sample means
- Mean ride duration on rainy days vs mean ride duration on dry days
>
>
> ##### All Trips
- $H$o : Mean Trip Duration on Rainy Days = Mean Trip Duration on Dry Days
- $H$a : Mean Trip Duration on Rainy Days ≠ Mean Trip Duration on Dry Days
>
> ##### Customer Trips
- $HC$o : Customer Mean Trip Duration on Rainy Days = Customer Mean Trip Duration on Dry Days
- $HC$a : Customer Mean Trip Duration on Rainy Days ≠ Customer Mean Trip Duration on Dry Days
>
> ##### Subscriber Trips
- $HS$o : Subscriber Mean Trip Duration on Rainy Days = Subscriber Mean Trip Duration on Dry Days
- $HS$a : Subscriber Mean Trip Duration on Rainy Days ≠ Subscriber Mean Trip Duration on Dry Days

### Results:

> ##### All Trips - Mean trip durations on rainy days are not equal to mean trip durations on dry days
- T Statistic <b>-21.3507</b> 
- P Value <b>4.1139e-101</b> which is well below the 0.05 threshhold thus we <b> reject</b> the $H$o

> ##### Customer Trips - Mean trip durations on rainy days are equal mean trip durations on dry days
- T Statistic <b>-0.7404</b> 
- P Value <b>0.4591</b> which is above the 0.05 threshhold thus we <b>can not reject</b> the $HC$o

> ##### Subscriber Trips - Mean trip durations on rainy days are not equal to mean trip durations on dry days
- T Statistic <b>-11.5109</b> 
- P Value <b>1.1706e-30</b> which is well below the 0.05 threshhold thus we <b>reject</b> the $HS$o

In [ ]:
# All Trips
rainy_data = rainy_trips.duration_minutes
dry_data = dry_trips.duration_minutes

# Customer Trips Only
customer_rainy_data = customer_rainy_trips.duration_minutes
customer_dry_data = customer_dry_trips.duration_minutes

# Subscriber Trips Only
subscriber_rainy_data = subscriber_rainy_trips.duration_minutes
subscriber_dry_data = subscriber_dry_trips.duration_minutes

In [ ]:
def calculate_stats(data1, data2):

    # means
    data1_mean = data1.mean()
    data2_mean = data2.mean()
    diff_mean = data1_mean - data2_mean
    print('Diff of means:\t\t', diff_mean)

    # calculate t statistic and p value with scipy
    t, p = stats.ttest_ind(data1, data2)
    print('T Test')
    print('\tt statistic:\t\t', t)
    print('\tp value:\t\t', p)
    print('')
    u, p2 = stats.mannwhitneyu(data1, data2)
    print('MannWhitneyU Test')
    print('\tu statistic:\t\t', u)
    print('\tp value:\t\t', p2)

In [ ]:
print('All Trips')
calculate_stats(rainy_data, dry_data)
print()
print('-' * 40)
print('Customer Trips')
calculate_stats(customer_rainy_data, customer_dry_data)
print()
print('-' * 40)
print('Subscriber Trips')
calculate_stats(subscriber_rainy_data, subscriber_dry_data)

In [ ]:
# plt.subplots(figsize=(12,6))
# sns.swarmplot(x="trip_dow", y="start_hour", hue="start_hour", data=customer_rainy_trips);

In [ ]:
customer_rainy_trips.info()

In [ ]:
sns.jointplot('start_terminal', 'end_terminal', data=customer_rainy_trips, kind='reg')
plt.show()

In [ ]:
# sns.jointplot('start_terminal', 'end_terminal', data=customer_rainy_trips[customer_rainy_trips['start_terminal'] == customer_rainy_trips['end_terminal']], kind='reg')
# plt.show()

# sns.jointplot('start_terminal', 'end_terminal', data=customer_rainy_trips[customer_rainy_trips['start_terminal'] != customer_rainy_trips['end_terminal']], kind='reg')
# plt.show()

In [ ]:
# sns.jointplot('end_terminal', 'end_hour', data=customer_rainy_trips, kind='reg')
# plt.show()

## 2. Does Rain have an Impact on trip distance for Customers, or Subscribers?

> A <b>Two Sample T Test</b> is appropriate for this problem as we are trying to see a difference between two sample means
- Mean ride duration on rainy days vs mean ride duration on dry days
>
>
> ##### All Trips
- $H$o : Mean Trip Distance on Rainy Days = Mean Trip Distance on Dry Days
- $H$a : Mean Trip Distance on Rainy Days ≠ Mean Trip Distance on Dry Days
>
> ##### Customer Trips
- $HC$o : Customer Mean Trip Distance on Rainy Days = Customer Mean Trip Distance on Dry Days
- $HC$a : Customer Mean Trip Distance on Rainy Days ≠ Customer Mean Trip Distance on Dry Days
>
> ##### Subscriber Trips
- $HS$o : Subscriber Mean Trip Distance on Rainy Days = Subscriber Mean Trip Distance on Dry Days
- $HS$a : Subscriber Mean Trip Distance on Rainy Days ≠ Subscriber Mean Trip Distance on Dry Days

### Results:

> ##### All Trips - Mean trip durations on rainy days are not equal to mean trip durations on dry days
- T Statistic <b>-6.2193</b> 
- P Value <b>4.9967e-10</b> which is well below the 0.05 threshhold thus we <b> reject</b> the $H$o

> ##### Customer Trips - Mean trip durations on rainy days are equal mean trip durations on dry days
- T Statistic <b>0.08450</b> 
- P Value <b>0.9327</b> which is above the 0.05 threshhold thus we <b>can not reject</b> the $HC$o

> ##### Subscriber Trips - Mean trip durations on rainy days are not equal to mean trip durations on dry days
- T Statistic <b>-7.1311</b> 
- P Value <b>9.9694e-13</b> which is well below the 0.05 threshhold thus we <b>reject</b> the $HS$o


In [ ]:
# All Trips
rainy_data = rainy_trips.distance_km
dry_data = dry_trips.distance_km

# Customer Trips Only
customer_rainy_data = customer_rainy_trips.distance_km
customer_dry_data = customer_dry_trips.distance_km

# Subscriber Trips Only
subscriber_rainy_data = subscriber_rainy_trips.distance_km
subscriber_dry_data = subscriber_dry_trips.distance_km

In [ ]:
# All Trips
print('rainy_data:\t\t ', len(rainy_data))
print('dry_data:\t\t', len(dry_data))

# Customer Trips Only
print('customer_rainy_data:\t  ', len(customer_rainy_data))
print('customer_dry_data:\t', len(customer_dry_data))

# Subscriber Trips Only
print('subscriber_rainy_data:\t ', len(subscriber_rainy_data))
print('subscriber_dry_data:\t', len(subscriber_dry_data))

In [ ]:
print('All Trips')
calculate_stats(rainy_data, dry_data)
print()
print('-' * 40)
print('Customer Trips')
calculate_stats(customer_rainy_data, customer_dry_data)
print()
print('-' * 40)
print('Subscriber Trips')
calculate_stats(subscriber_rainy_data, subscriber_dry_data)

## Weekend vs weekday subscribers

In [ ]:
trips.info()

In [ ]:
trips.describe()

In [ ]:
trips.head()

In [ ]:
weekday_trips = trips[trips['start_date'].dt.weekday < 5]
weekend_trips = trips[trips['start_date'].dt.weekday >= 5]

In [ ]:
# number of trips by weekday/weekend and subscriber type
weekday_customer_trips   = weekday_trips[weekday_trips.subscriber_type == 'Customer']
weekend_customer_trips   = weekend_trips[weekend_trips.subscriber_type == 'Customer']
weekday_subscriber_trips = weekday_trips[weekday_trips.subscriber_type != 'Customer']
weekend_subscriber_trips = weekend_trips[weekend_trips.subscriber_type != 'Customer']

# number of days with customer weekend trips
weekday_customer_trip_days =   len(pd.unique(weekday_customer_trips.trip_date))
weekend_customer_trip_days =   len(pd.unique(weekend_customer_trips.trip_date))
weekday_subscriber_trip_days = len(pd.unique(weekday_subscriber_trips.trip_date))
weekend_subscriber_trip_days = len(pd.unique(weekend_subscriber_trips.trip_date))

# trips per day for each group
weekday_customer_tpd   = len(weekday_customer_trips)   / weekday_customer_trip_days
weekend_customer_tpd   = len(weekend_customer_trips)   / weekend_customer_trip_days
weekday_subscriber_tpd = len(weekday_subscriber_trips) / weekday_subscriber_trip_days
weekend_subscriber_tpd = len(weekend_subscriber_trips) / weekend_subscriber_trip_days

customer_ratio   = weekday_customer_tpd / weekend_customer_tpd * 100.
subscriber_ratio = weekday_subscriber_tpd / weekend_subscriber_tpd  * 100.



print('-' * 80)
print('weekday_customer_trip_days:\t\t   ', weekday_customer_trip_days)
print('weekend_customer_trip_days:\t\t   ', weekend_customer_trip_days)
print('weekday_subscriber_trip_days:\t\t   ', weekday_subscriber_trip_days)
print('weekend_subscriber_trip_days:\t\t   ', weekend_subscriber_trip_days)
print('-' * 80)
print('Number of Weekday Customer Trips):\t ', len(weekday_customer_trips))
print('Number of Weekend Customer Trips):\t ', len(weekend_customer_trips))
print('Number of Weekday Subscriber Trips):\t', len(weekday_subscriber_trips))
print('Number of Weekend Subscriber Trips):\t ', len(weekend_subscriber_trips))
print('-' * 80)
print('weekday_customer Trips Per Day:\t\t    ', weekday_customer_tpd)
print('weekend_customer Trips Per Day:\t\t   ', weekend_customer_tpd)
print('weekday_subscriber Trips Per Day:\t   ', weekday_subscriber_tpd)
print('weekend_subscriber Trips Per Day:\t   ', weekend_subscriber_tpd)
print('-' * 80)
print('Customer Weekday/Weekend Ratio:\t\t   ', customer_ratio)
print('Subscriber Weekday/Weekend Ratio:\t  ', subscriber_ratio)
print('-' * 80)

In [ ]:
weekday_customer_trips   = weekday_trips[weekday_trips.subscriber_type == 'Customer']
weekend_customer_trips   = weekend_trips[weekend_trips.subscriber_type == 'Customer']
weekday_subscriber_trips = weekday_trips[weekday_trips.subscriber_type != 'Customer']
weekend_subscriber_trips = weekend_trips[weekend_trips.subscriber_type != 'Customer']



In [ ]:
# duration data
day_cust_dur  = weekday_customer_trips.duration_minutes
end_cust_dur  = weekend_customer_trips.duration_minutes
day_sub_dur  = weekday_subscriber_trips.duration_minutes
end_sub_dur  = weekend_subscriber_trips.duration_minutes

# distance data
day_cust_dist = weekday_customer_trips.distance_km
end_cust_dist = weekend_customer_trips.distance_km
day_sub_dist = weekday_subscriber_trips.distance_km
end_sub_dist = weekend_subscriber_trips.distance_km


In [ ]:

print('-' * 40)
print('Customer Trip Duration')
calculate_stats(day_cust_dur, end_cust_dur)
print()
print('-' * 40)
print('Subscriber Trip Duration')
calculate_stats(day_sub_dur, end_sub_dur)
print()
print('-' * 40)

In [ ]:
print('-' * 40)
print('Customer Trip Distance')
calculate_stats(day_cust_dist, end_cust_dist)
print()
print('-' * 40)
print('Subscriber Trip Distance')
calculate_stats(day_sub_dist, end_sub_dist)
print()
print('-' * 40)

In [ ]:
customer_trips = trips[trips.subscriber_type == 'Customer']
subscriber_trips = trips[trips.subscriber_type == 'Subscriber']

In [ ]:
_trips= stats.normaltest(trips.start_hour.sort_values())
print('_trips:\t\t', _trips)
_customer= stats.normaltest(customer_trips.start_hour.sort_values())
print('_customer:\t', _customer)
_subscriber= stats.normaltest(subscriber_trips.start_hour.sort_values())
print('_subscriber:\t', _subscriber)

In [ ]:
# calculate_stats(rainy_data, dry_data)
stats.mannwhitneyu(rainy_data, dry_data)

In [ ]:
stats.mannwhitneyu(dry_data, rainy_data)

In [ ]:
stats.mannwhitneyu(customer_rainy_data, customer_dry_data)

In [ ]:
stats.mannwhitneyu(subscriber_rainy_data, subscriber_dry_data)

In [ ]:
am_trips = trips[trips.start_hour < 12]
pm_trips = trips[trips.start_hour >= 12]

In [ ]:
am_start_terminal_trips = am_trips.groupby('start_terminal').duration.count()
pm_start_terminal_trips = pm_trips.groupby('start_terminal').duration.count()
calculate_stats(am_start_terminal_trips, pm_start_terminal_trips)

In [ ]:
am_end_terminal_trips = am_trips.groupby('end_terminal').duration.count()
pm_end_terminal_trips = pm_trips.groupby('end_terminal').duration.count()
calculate_stats(am_end_terminal_trips, pm_end_terminal_trips)

In [ ]:
plt.subplot(2,2,1)
am_start_terminal_trips.plot(kind='bar', figsize=(12,6))

plt.subplot(2,2,2)
pm_start_terminal_trips.plot(kind='bar')

plt.subplot(2,2,3)
am_end_terminal_trips.plot(kind='bar')

plt.subplot(2,2,4)
pm_end_terminal_trips.plot(kind='bar')


plt.tight_layout()
plt.show()

In [ ]:

customer_am_start_terminal_trips = am_trips[am_trips.subscriber_type == 'Customer'].groupby('start_terminal').duration.count()
customer_am_end_terminal_trips = am_trips[am_trips.subscriber_type == 'Customer'].groupby('end_terminal').duration.count()

customer_pm_start_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Customer'].groupby('start_terminal').duration.count()
customer_pm_end_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Customer'].groupby('end_terminal').duration.count()

plt.subplot(4,1,1)
customer_am_start_terminal_trips.plot(kind='bar', figsize=(12,6))

plt.subplot(4,1,2)
customer_pm_start_terminal_trips.plot(kind='bar')

plt.subplot(4,1,3)
customer_am_end_terminal_trips.plot(kind='bar')

plt.subplot(4,1,4)
customer_pm_end_terminal_trips.plot(kind='bar')


plt.tight_layout()
plt.show()

In [ ]:
subscriber_am_start_terminal_trips = am_trips[am_trips.subscriber_type == 'Subscriber'].groupby('start_terminal').duration.count()
subscriber_am_end_terminal_trips = am_trips[am_trips.subscriber_type == 'Subscriber'].groupby('end_terminal').duration.count()

subscriber_pm_start_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Subscriber'].groupby('start_terminal').duration.count()
subscriber_pm_end_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Subscriber'].groupby('end_terminal').duration.count()


ax = subscriber_am_start_terminal_trips.plot(kind='line', figsize=(12,6))
subscriber_pm_start_terminal_trips.plot(kind='line', ax=ax)

subscriber_am_end_terminal_trips.plot(kind='line', ax=ax)
subscriber_pm_end_terminal_trips.plot(kind='line', ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
subscriber_am_start_terminal_trips = am_trips[am_trips.subscriber_type == 'Subscriber'].groupby('start_terminal').duration.count()
subscriber_am_end_terminal_trips = am_trips[am_trips.subscriber_type == 'Subscriber'].groupby('end_terminal').duration.count()

subscriber_pm_start_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Subscriber'].groupby('start_terminal').duration.count()
subscriber_pm_end_terminal_trips = pm_trips[pm_trips.subscriber_type == 'Subscriber'].groupby('end_terminal').duration.count()

station_diff = pd.DataFrame()
station_diff['am_start_trips'] = subscriber_am_start_terminal_trips
station_diff['am_end_trips'] = subscriber_am_end_terminal_trips
station_diff['am_diff'] = subscriber_am_start_terminal_trips - subscriber_am_end_terminal_trips


station_diff['pm_start_trips'] = subscriber_pm_start_terminal_trips
station_diff['pm_end_trips'] = subscriber_pm_end_terminal_trips
station_diff['pm_diff'] = subscriber_pm_start_terminal_trips - subscriber_pm_end_terminal_trips

station_diff['removed_overnight'] = station_diff['am_diff'] - station_diff['pm_diff']

In [ ]:
ax = station_diff.am_diff.plot(kind='bar', figsize=(12,6))
station_diff.pm_diff.plot(kind='bar', ax=ax, color='r')
plt.show()

In [ ]:
station_diff.removed_overnight.plot(kind='bar', figsize=(12,6))
plt.show()

In [ ]:
sns.jointplot(x='am_start_trips', y='pm_end_trips', data=station_diff, kind='reg')
plt.show()

In [ ]:
sns.jointplot(x='am_end_trips', y='pm_start_trips', data=station_diff, kind='reg')
plt.show()

In [ ]:
hourly_stations = pd.DataFrame()
hourly_stations_diff = pd.DataFrame()

for i in range(0, 24):
    _trips = trips[trips.start_hour == i]
    _start_terminal = _trips.groupby('start_terminal').duration.count()
    _end_terminal = _trips.groupby('end_terminal').duration.count()
    
    start_string = 'hour%s_end' % str(i)
    end_string = 'hour%s_start' % str(i)
    diff_string = 'hour%s' % str(i)
    hourly_stations[end_string] = _end_terminal
    hourly_stations[start_string] = _start_terminal
    hourly_stations_diff[diff_string] = _start_terminal - _end_terminal


In [ ]:
hourly_stations.head(3)

In [ ]:
hourly_stations_diff.info()

In [ ]:
hourly_stations_diff.head(3)